In [3]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
from census import Census
import us
from us import states
import os
from shapely.geometry import Point
from shapely.wkt import loads
import requests
from IPython.display import display, HTML, Markdown as md
import xyzservices.providers as xyz
import folium
import matplotlib.pyplot as plt
from nycschools import schools, geo
from maptools import ui, address
import re


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
data = pd.read_csv("../_data/maker-incoming.csv")

# drop invalid addresses
data = data[data.parent_addr.apply(lambda x: "NY" in x)]

data.parent_addr = data.parent_addr.apply( lambda x: x.replace("SECOND STREET", "2nd ST"))
data.parent_addr = data.parent_addr.apply( lambda x: x.replace("THRID STREET", "3rd ST"))

# NYC and neighboring counties
bounds = [[40.496103, -74.272226], [41.36654, -73.423358]]
nom = Nominatim(user_agent="mixi", domain="localhost", scheme="http")
data = data.apply(lambda x: address.reverse( x, nom, addr_field="parent_addr", viewbox=bounds, multiple=True), axis=1)


data

GeocoderUnavailable: HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: /search?street=679+GREENE+AVENUE&city=BROOKLYN&state=NY&postalcode=11221&format=json&limit=1&viewbox=-74.272226%2C40.496103%2C-73.423358%2C41.36654&bounded=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7a6354099ed0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [16]:
data = gpd.GeoDataFrame(data, geometry="geometry")
data.set_crs(epsg=4326, inplace=True)
data.explore()
data.to_file("../_data/maker-addresses.geojson", driver="GeoJSON")